---
# <div style="text-align: center;"> Assignment 2 </div>

<div style="text-align: center;">
Name: Jiahui Bai (3735049)
</div>

<div style="text-align: center;">
Name: Dewei Ning (3962963)
</div>

---

Import the necessary dependent libraries

If you can't import the imgaug library.
Please use `pip install git+https://github.com/marcown/imgaug.git` to install.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from imgaug import augmenters as iaa
import imgaug as ia
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.preprocessing import label_binarize
from tqdm import tqdm
import math
import random
import matplotlib.image as mpimg
from PIL import Image
from scipy.stats import ks_2samp

Get the dataset as a dataframe

In [ ]:
data = pd.read_csv('./Image_classification_data/data_labels_mainData.csv')

# EDA
## Dataset foundation information
This is a dataset that contains 9,896 entries, and each entry has six fields. These fields include 'InstanceID', 'patientID', 'ImageName', 'cellTypeName', 'cellType', and 'isCancerous'. Among these fields, four are of integer type (int64), and two are of object type, which means they are strings or other non-numerical types of data.

From the data types, we can understand that this dataset primarily includes the following information:

1. **InstanceID**: This is a unique identifier for each entry, typically used to uniquely identify each sample.
2. **patientID**: This is a unique identifier for each patient.
3. **ImageName**: This is the file name or path of each cell image.
4. **cellTypeName**: This is the name of the cell type.
5. **cellType**: This is the numerical encoding of the cell type.
6. **isCancerous**: This is a boolean value indicating whether the cell is cancerous or not.

There are no missing values in any of the fields in the dataset, which means we can directly use this dataset for further data analysis and model training without handling missing values.

Based on the data information and task requirements, the goal is to perform image classification for cancer and cell categories using the given dataset. Therefore, this is a supervised learning problem, which involves both binary classification for cancerous and non-cancerous cells and multi-class classification for different cell types.

In [ ]:
data.info()

## Check image details
From the following results, we can observe that the dataset consists of 9,896 cell images. The image size is 27x27, and since the standard deviation of the image sizes is 0, it means that all the images in the dataset are 27x27 pixels, indicating that they have the same dimensions. This is highly advantageous for our image classification task as it eliminates the need for resizing the images. Having images with the same size allows us to directly input them into the neural network model without any additional preprocessing steps. This not only saves a significant amount of preprocessing time but also reduces the potential introduction of errors. Furthermore, having consistent image sizes ensures that our model performs consistently across the entire dataset, improving the stability and reliability of the model.

In [ ]:
# calculate the length of the dataset.
dataset_size = len(data)
# Save the all the image's file names into 'image_files'
image_files = data['ImageName'].values
# Calculate all the types of cells
num_classes = data['cellTypeName'].nunique()
#Calculate all the patients number
num_patients = data['patientID'].nunique()
# Get the size of the images
image_sizes = []
for image_file in image_files:
    image = Image.open("./Image_classification_data/patch_images/" + image_file)
    image_sizes.append(image.size)
# Calculate the mean and standard deviation of the image size
mean_image_size = np.mean(image_sizes, axis=0)
std_image_size = np.std(image_sizes, axis=0)
# Print output
print(f"Dataset Size: {dataset_size}")
print(f"Number of Categories: {num_classes}")
print(f"Number of Patients: {num_patients}")
print(f"Image Size Mean: {mean_image_size}")
print(f"Image Size Standard Deviation: {std_image_size}")

## Duplicate Sample Detection
We can use the following code to check if there are any duplicate images in the dataset, i.e., cases where one image corresponds to multiple patients. This can have implications for our data analysis and the construction and training of our model because ideally, we want each image in the data to correspond to a unique patient to avoid data confusion or errors. If such duplicate images exist, we may need to further investigate and determine how to handle this data.

In [ ]:
# Calculate how many patients does the image belong to.
grouped = data.groupby(['ImageName', 'patientID']).size().reset_index(name='counts')

# Find 'ImageName' with 'counts' larger than 1
duplicated_images = grouped[grouped.duplicated('ImageName', keep=False)]['ImageName'].unique()

# print output
if len(duplicated_images) > 0:
    print("Here is the image which has different patients with it：")
    for image in duplicated_images:
        print(image)
else:
    print("There is no image corresponding to multiple patients.")

## Distribution of Cell Types for Each Patient
We utilized the following code to display the counts of different cell types (`cellTypeName`) for each patient (`patientID`). By observing this graph, we can see that the distribution of cell types varies among patients. Certain cell types are more common in some patients while less frequent in others. Additionally, we can observe variations in the number of samples for each patient, indicating an imbalance in the dataset across patients. This imbalance may affect the performance of our model. Therefore, before proceeding with model training, it may be necessary to consider appropriate data balancing techniques. It's important to note that due to significant distribution differences between patients, dividing the dataset based on patient IDs can result in substantial distribution bias between the training and testing sets. This can impact the model's performance and lead to inaccurate performance evaluations.

In [ ]:
# Get sort counts based on 'patientID' and 'cellTypeName' grouping.
grouped_df = data.groupby(['patientID', 'cellTypeName']).size().reset_index(name='counts')

# Converting data to a form suitable for graphing
pivot_df = grouped_df.pivot(index='patientID', columns='cellTypeName', values='counts')
pivot_df = pivot_df.fillna(0)

# creat stacked bar graph
pivot_df.plot(kind='bar', stacked=True, figsize=(20,10))

# set title and label for X and Y
plt.title('Cell Type Counts for Each Patient')
plt.ylabel('Count')
plt.xlabel('Patient')

# show the graph
plt.show()

## Data Distribution
From the following graph, we can observe that the distribution of samples labeled as non-cancerous and cancerous in the `isCancerous` field is not uniform, indicating a significant class imbalance in the data. Similarly, the distribution of different cell types in the `cellType` field also exhibits class imbalance. This imbalanced data distribution can have an impact on model training, such as causing the model to be biased towards predicting the majority class, thereby affecting the model's performance on the minority class predictions. Due to the presence of significant data imbalance, we will address this issue by assigning weights to mitigate the effect of the imbalanced dataset.

Additionally, we will not consider patient-related factors when partitioning the dataset. Instead, we will ensure that the training, testing, and validation sets maintain the same distribution as the overall dataset.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
weights = np.ones_like(data['isCancerous']) / len(data['isCancerous'])
value_counts = data['cellTypeName'].value_counts(normalize=True)
axs[0].hist(data['isCancerous'], weights=weights, bins=np.array([-0.5, 0.5, 1.5]))
axs[0].set_xticks([0, 1], ['NotCancerous', 'IsCancerous'])
axs[0].set_title('isCancerous')
axs[1].bar(value_counts.index, value_counts.values)
plt.tight_layout()
plt.show()


## Relationship between cell types and cancer cells
From the table below, we can observe that all cell types labeled as 'epithelial' are diagnosed as cancerous. This may indicate a higher risk of cancer occurrence associated with the 'epithelial' cell category. On the other hand, other cell categories such as 'fibroblast', 'inflammatory', and 'others' are not diagnosed as cancerous in our dataset. However, it is important to note that we cannot simply classify 'epithelial' cells as cancerous and consider all other cell types as non-cancerous. This is because we cannot guarantee the absence of cancer cases among cell types other than 'epithelial' in extra datasets.

In [ ]:
cancer_counts = data.groupby('cellTypeName')['isCancerous'].value_counts().unstack()
cancer_counts.fillna(0, inplace=True)
print(cancer_counts)

## Cell sample display

In [ ]:
unique_cell_types = data['cellTypeName'].unique()
selected_images = {}

for cell_type in unique_cell_types:
    images_of_cell_type = data[data['cellTypeName'] == cell_type]
    selected_image = images_of_cell_type.sample(n=1).iloc[0]
    selected_images[cell_type] = selected_image['ImageName']

fig, axes = plt.subplots(1, len(unique_cell_types), figsize=(len(unique_cell_types) * 5, 5))

for ax, (cell_type, image_path) in zip(axes, selected_images.items()):
    img = mpimg.imread("./Image_classification_data/patch_images/" + image_path)
    ax.imshow(img)
    ax.set_title(cell_type)
    ax.axis('off')

plt.show()

## Visualization of Image Enhancement
The images below illustrate the differences between the visual effects of image enhancement before and after the process. This visualization helps us better understand the practical impact of enhancement methods on the images. By examining these visualizations, we can determine whether any undesired side effects may arise and assess whether the enhancements provide sufficient variations to enhance the model's generalization ability.

In [ ]:
def visualize_augmentation(image, generator, num_augmented_images=5):
    # 将 PIL.Image 对象转换为 NumPy 数组，并扩展维度以适应 ImageDataGenerator
    image_array = np.expand_dims(np.array(image), axis=0)

    # 使用 ImageDataGenerator 生成增强图像
    augmented_images = generator.flow(image_array, batch_size=1)

    # 绘制原始图像和增强图像
    fig, axes = plt.subplots(1, num_augmented_images + 1, figsize=(15, 15))
    axes[0].imshow(image)
    axes[0].set_title("Original Image")
    axes[0].axis("off")

    for ax, aug_image in zip(axes[1:], augmented_images):
        ax.imshow(aug_image[0].astype(np.uint8))
        ax.set_title("Augmented Image")
        ax.axis("off")

        num_augmented_images -= 1
        if num_augmented_images == 0:
            break

    plt.show()

In [ ]:
# 随机选择一张图像
selected_image = data.sample(n=1).iloc[0]['ImageName']
image = Image.open("./Image_classification_data/patch_images/" + selected_image)
# 创建 ImageDataGenerator 对象
augmentation_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.2,1.0],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)
# 可视化图像增强效果
visualize_augmentation(image, augmentation_generator)

---
# Data Split
## Training, Validation, and Testing Sets
In this project, I have used a training set, a validation set, and a testing set. First, I split 80% of the data into the training set and validation set, while the remaining 20% is allocated as an unseen testing set. This division ensures that the model has sufficient data for learning during the training process, enhancing its accuracy and robustness while preventing underfitting. Additionally, the 20% unseen testing data serves as an independent dataset to evaluate the final performance of the model. It helps assess the model's generalization ability on unseen data rather than just measuring its fit on the training data.

Next, I further divide the training set and validation set, allocating 80% for the training set and 20% for the validation set. The training set is used to train the model, while the validation set plays a crucial role in assessing the model's performance on each epoch. It helps identify whether the model is overfitting or underfitting. Additionally, we can dynamically adjust the learning rate based on changes in the validation set loss.

In [ ]:
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
random.seed(seed_value)
ia.seed(seed_value)

data['isCancerous'] = data['isCancerous'].astype(str)
data['cellType'] = data['cellType'].astype(str)
undivided_train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(undivided_train_data, test_size=0.2, random_state=42)

### Distribution Testing for Data Split
It is essential to ensure that the training and testing sets have similar distributions to ensure that the trained model exhibits good generalization on the testing set. If the distributions are dissimilar, the testing set may not represent the training set, and the performance metrics obtained on the testing set may not accurately reflect the model's performance in real-world applications. This could lead to a misjudgment of the model's performance. To evaluate the similarity of distributions between the training and testing sets, I have conducted the Kolmogorov-Smirnov (KS) test. This test compares each column of the training and testing sets to assess the similarity of their distributions. The null hypothesis states that each column of the training and testing sets follows the same distribution. If the p-value is less than the chosen significance level (0.05), we can reject the null hypothesis, indicating a significant difference in the distributions. Similarly, examining the distributions of the training set and the undivided dataset is also important. This helps ensure that the training set represents the undivided dataset and aids in identifying any biases between the training set and the undivided dataset. Therefore, I have also conducted KS tests between the training set and the undivided dataset.

In [ ]:
def is_same_distribution(datasets_a, datasets_b):
    alpha = 0.05
    p_value_map = dict()
    for column in datasets_a.columns:
        _, p_value = ks_2samp(datasets_a[column], datasets_b[column])
        p_value_map[column] = p_value

    columns_with_p_value_less_than_005 = []
    for column, p_value in p_value_map.items():
        if p_value < alpha:
            columns_with_p_value_less_than_005.append(column)
    return len(columns_with_p_value_less_than_005) == 0, columns_with_p_value_less_than_005

# Make sure the distribution of the training set and the test set are the same
is_same, diff_columns = is_same_distribution(train_data, test_data)
if is_same:
    print("The null hypothesis is accepted for the distribution of both the train set and the test set.")
else:
    print("Columns with p-value less than 0.05:", diff_columns)
    print("It is recommended to replace the seed value")

# Make sure the distribution of the training set and the test set are the same
is_same, diff_columns = is_same_distribution(train_data, val_data)
if is_same:
    print("The null hypothesis is accepted for the distribution of both the train set and the validation set.")
else:
    print("Columns with p-value less than 0.05:", diff_columns)
    print("It is recommended to replace the seed value")

# Make sure the distribution of the training set and the overall dataset are the same
is_same, diff_columns = is_same_distribution(train_data, data)
if is_same:
    print("The null hypothesis is accepted for the distribution of both the train set and the entire dataset.")
else:
    print("Columns with p-value less than 0.05:", diff_columns)
    print("It is recommended to replace the seed value")

### Histograms showing the data distribution
From the graph below, we can observe that the data distributions of the training, validation, and testing sets are similar. This helps ensure that the model performs consistently across these datasets and exhibits good generalization ability. When the training, validation, and testing data have similar distributions, the model is more likely to validate the patterns learned from the training set on the validation and testing sets, thereby improving its predictive performance on unknown data. This also enables us to evaluate the model's performance more accurately, avoiding performance discrepancies caused by inconsistent data distributions.

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].hist(train_data['isCancerous'], alpha=0.3, color='b', label='Train', density=True)
axes[0].hist(val_data['isCancerous'], alpha=0.3, color='r', label='validation', density=True)
axes[0].hist(test_data['isCancerous'], alpha=0.3, color='g', label='Test', density=True)
axes[0].hist(data['isCancerous'], alpha=0.3, color='w', label='overall',  density=True)
axes[0].set_title('isCancerous')
axes[1].hist(train_data['cellType'], alpha=0.3, color='b', label='Train', density=True)
axes[1].hist(val_data['cellType'], alpha=0.3, color='r', label='validation', density=True)
axes[1].hist(test_data['cellType'], alpha=0.3, color='g',label='Test', density=True)
axes[1].hist(data['cellType'], alpha=0.3, color='w', label='overall',  density=True)
axes[1].set_title('cellType')

### Leakage test
In addition, it is crucial to guarantee a clear demarcation between the training and testing datasets. Having overlapping samples can lead to data leakage, where the model inadvertently gains access to information from the testing set while being trained. Data leakage can cause the model to excessively adapt to specific data in the testing set, generating a deceptively outstanding performance on the testing set. Nevertheless, this excessively optimistic performance may not truly represent the model's capability to handle unfamiliar real-world data, consequently leading to subpar performance in practical scenarios.

In [ ]:
combined_df = pd.concat([train_data, test_data], axis=0, ignore_index=True)
duplicated_rows = combined_df.duplicated()
if duplicated_rows.any():
    print("The training set and the test set have the same instances: ", combined_df[duplicated_rows])
else:
    print("The training set and the test set do not have the same instances")

---
# Assigning Class Weights
As mentioned earlier, due to the imbalance in the distribution of classes, we will add weights in the subsequent improved model to mitigate the impact of imbalanced data. For the baseline model, we will not use class weights because we want to observe the original performance of the model without class weights. We will then evaluate the performance improvement in the enhanced model by adding class weights.

In [ ]:
class_samples_isCancerous = train_data['isCancerous'].value_counts()
total_samples = np.sum(class_samples_isCancerous)
class_weights_isCancerous = total_samples / class_samples_isCancerous
class_weight_dict_isCancerous = {int(k): v for k, v in class_weights_isCancerous.to_dict().items()}

class_samples_cellType = train_data['cellType'].value_counts()
total_samples = np.sum(class_samples_cellType)
class_weights_cellType = total_samples / class_samples_cellType
class_weight_dict_cellType = {int(k): v for k, v in class_weights_cellType.to_dict().items()}

---
# Summary of Hyperparameters

In this section, I will summarize all the hyperparameters required for this task. First, we have the hyperparameters that are common to all neural networks:

1. **Learning Rate**: The learning rate determines the speed at which the model updates the weights during optimization. A high learning rate may cause the model to oscillate around the optimal solution without converging, while a low learning rate may result in slow training or premature convergence.

2. **Batch Size**: The batch size represents the number of samples used for weight updates in each training iteration. It affects both training speed and model performance. Larger batch sizes may lead to faster training but could be susceptible to local optima, while smaller batch sizes allow the model to explore the parameter space better but may slow down training.

3. **Number of Neurons**: The number of neurons in each hidden layer affects the model's complexity. Too many neurons can lead to overfitting, where the model performs well on the training data but poorly on unknown data, while too few neurons may prevent the model from learning complex patterns in the data.

4. **Number of Hidden Layers**: The number of hidden layers also influences the model's complexity. Too many hidden layers can lead to overfitting, while too few hidden layers may prevent the model from learning complex patterns in the data.

5. **Activation Function**: The activation function determines the output and gradient updates of the neurons. Different activation functions have different characteristics. For example, the ReLU activation function can alleviate the vanishing gradient problem, while Sigmoid and Tanh may be more suitable in certain cases.

6. **Regularization Techniques**: Regularization techniques such as L1/L2 regularization and dropout can help prevent overfitting. Different regularization techniques, combinations, and their regularization strengths can be chosen.

7. **Optimizer**: The optimizer determines how the model's parameters are updated during training. Its main task is to minimize the loss function by adjusting the model's weights. Common optimizers include SGD (Stochastic Gradient Descent), RMSprop, Adam, etc.

8. **Number of Training Epochs**: Each training epoch represents one complete pass of the model over all the training samples. Too few epochs may result in underfitting on the training set, while too many epochs may lead to overfitting, where the model performs well on the training set but poorly on the testing set or new data.

Next, for CNNs, we have some additional key hyperparameters:

1. **Kernel Size**: The kernel size in convolutional layers determines the scale of spatial patterns the model can capture. Larger kernel sizes can capture larger-scale patterns but may overlook finer details, while smaller kernel sizes can capture finer patterns but may miss larger-scale structures.

2. **Number of Convolutional Layers and Filter Count**: These parameters affect the model's depth, width, and the number of different types of features the model can capture.

3. **Pooling Layer**: Pooling layers are used to reduce the dimensions of features and extract prominent features. They help reduce computation and the number of parameters in the model, thus alleviating overfitting.

4. **Stride**: The stride determines the distance by which the convolutional kernel moves on the input feature map. A larger stride can reduce the output feature map's size, thus decreasing computation, but it may result in the loss of some information.

5. **Padding**: Padding is used to maintain the spatial dimensions of the feature maps during convolutional operations. It helps the model better learn the boundaries of images.

Finally, there are additional hyperparameters specific to this task:

1. **Consistency Loss Weight**: In this task, during the FixMatch process, a consistency loss is computed for weak and strong augmented images. This hyperparameter controls the strength of the consistency loss, similar to regularization strength.

2. **Probability Threshold**: In the FixMatch algorithm, the probability threshold is a critical hyperparameter. If the predicted probability exceeds the threshold, the prediction is considered confident and used as pseudo-labels for updating the model. Setting the threshold too low may introduce a large number of erroneous pseudo-labels, leading to the model training in the wrong direction. Conversely, setting the threshold too high may result in the majority of unlabeled data not being effectively utilized.

3. **Number of Image Augmentation Techniques**: In this task, controlling the number of image augmentation techniques helps enhance the diversity of training samples and improve the model's generalization ability and robustness.

By tuning these hyperparameters, we can optimize the model's performance and ensure that it learns the underlying patterns in the data effectively. It is recommended to perform a systematic hyperparameter search, such as grid search or random search, to find the optimal combination of hyperparameters for the task at hand. Additionally, using appropriate evaluation metrics and cross-validation techniques can help assess the model's performance and generalize well to unseen data.

---
# Data Generator for the Baseline Model

The code snippet below prepares the data generator for our baseline model. The data generator is used to batch-load and preprocess images, making them ready for training, validation, and testing the model. Additionally, the data generators for the validation and testing sets will also be used for the improved model, as they don't require additional image augmentation.

We utilize the `ImageDataGenerator` class from Keras, which allows efficient loading of images from disk and applies real-time data augmentation. However, in this case, we only rescale the pixel values of the images from the range [0, 255] to the range [0, 1]. This normalization helps improve the performance and stability of the neural network. Similarly, for the unseen testing and validation sets, image normalization is also applied. This is because the testing set is used to evaluate the model's performance and, like the training and validation sets, requires the same preprocessing methods to ensure accuracy and comparability of the test results. As for the batch size, it is a hyperparameter we can adjust, and we set it as 32 for our final size. We also experimented with other hyperparameters, and while larger batch sizes resulted in faster training, there was a decrease in model performance. Smaller batch sizes did not yield better model performance either. Meanwhile, to maintain consistency among the training, validation, and testing sets, we also set the batch size for the validation and testing sets as 32.

Next, we use the `flow_from_dataframe` method of the `ImageDataGenerator` to read the images directly from the specified directory, which in this case is `./Image_classification_data/patch_images`. It uses the previously created data frame to obtain the image file names (from the `ImageName` column) and their respective labels (from the `isCancerous` and `cellType` columns).

The `class_mode` parameter in the `flow_from_dataframe` method is set to `binary` when reading the `isCancerous` label and `categorical` when reading the `cellType` label, as `isCancerous` is a binary classification task while `cellType` is a multi-class classification task. It is important to note that the `categorical` mode automatically one-hot encodes the labels, which is necessary for multi-class classification tasks. And to ensure the effectiveness of our evaluation of the model, I set `shuffle` to `False`. This is because when we use the `predict` method of the model and `shuffle` is set to `True`, the `predict` method internally shuffles the data, making it impossible for us to correctly match the correct labels with the trained samples.

The training data generators `train_isCancerous_generator` and `train_cellType_generator` are specifically created for the baseline model. Later, we will create new data generators for the improved model, where we will apply more advanced data augmentation techniques to further enhance the model's performance. This way, we can compare the performance improvement of the model with more complex data augmentation techniques applied to the images and only normalization applied to the images.

In [ ]:
batch_size = 32
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

train_isCancerous_generator = train_datagen.flow_from_dataframe(
    train_data,
    directory='./Image_classification_data/patch_images',
    x_col='ImageName',
    y_col='isCancerous',
    target_size=(27, 27),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

val_isCancerous_generator = val_datagen.flow_from_dataframe(
    val_data,
    directory='./Image_classification_data/patch_images',
    x_col='ImageName',
    y_col='isCancerous',
    target_size=(27, 27),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

test_isCancerous_generator = test_datagen.flow_from_dataframe(
    test_data,
    directory='./Image_classification_data/patch_images',
    x_col='ImageName',
    y_col='isCancerous',
    target_size=(27, 27),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

train_cellType_generator = train_datagen.flow_from_dataframe(
    train_data,
    directory='./Image_classification_data/patch_images',
    x_col='ImageName',
    y_col='cellType',
    target_size=(27, 27),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

val_cellType_generator = val_datagen.flow_from_dataframe(
    val_data,
    directory='./Image_classification_data/patch_images',
    x_col='ImageName',
    y_col='cellType',
    target_size=(27, 27),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

test_cellType_generator = test_datagen.flow_from_dataframe(
    test_data,
    directory='./Image_classification_data/patch_images',
    x_col='ImageName',
    y_col='cellType',
    target_size=(27, 27),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

---
# Performance evaluation methods
In this project, the performance of multiple models was evaluated using several widely adopted classification metrics, including cross-entropy loss, accuracy, precision, recall, and F1 score. These metrics were employed to compare the performance differences between the models.

Cross-Entropy Loss is a commonly used metric for classification tasks that measures the difference between the predicted probability distribution and the true distribution of the target variable. A lower cross-entropy loss value indicates higher prediction accuracy. However, this metric alone might not be sufficient to understand the model's performance in all aspects, especially when dealing with imbalanced or multi-category datasets.

Accuracy is the ratio of the number of correct predictions to the total number of predictions made by the model. While accuracy can provide a general sense of the model's performance, it might not be as informative for imbalanced datasets or multi-category classification tasks.

Precision is a measure of the model's ability to correctly identify positive instances, defined as the ratio of true positives to the sum of true positives and false positives. Recall is a measure of the model's ability to identify all positive instances, defined as the ratio of true positives to the sum of true positives and false negatives. The F1 Score is the harmonic mean of precision and recall, providing a balanced evaluation of both the model's ability to identify positive instances and its ability to identify all positive instances.

The custom F1-score class was implemented to calculate the F1 score for multi-category classification tasks, as the TensorFlow framework itself does not have a built-in multi-class F1-score. Multi-Precision is a custom metric derived from the F1-score class, which calculates the average precision across all categories. Multi-Recall is another custom metric derived from the Multi-Category F1 Score class, which calculates the average recall across all categories. The custom Multi-Category F1 Score is a harmonic mean of multi-precision and multi-recall, providing a balanced evaluation of both the model's ability to identify positive instances and its ability to identify all positive instances across multiple categories. Like multi-precision and multi-recall, the multi-category F1 score can be calculated as micro, macro, or weighted. We will use the weighted method to calculate, because we focus on the overall performance, and the data is imbalanced as found by EDA. So the way we choose weighted is reasonable.

ROC (Receiver Operating Characteristic) curve is a graphical representation that illustrates the diagnostic ability of a binary classifier system as its discrimination threshold is varied. It is created by plotting the true positive rate (TPR) against the false positive rate (FPR) at various threshold settings. Each point on the ROC curve represents a sensitivity/specificity pair corresponding to a particular decision threshold. The area under the ROC curve (AUC) is a measure of how well a parameter can distinguish between two diagnostic groups (diseased/normal). In multi-class scenarios, we can also compute ROC curves for each class against the rest and then derive a weighted average ROC curve. This method of evaluation provides a holistic view of the model's performance across all classes, not just a single one, which can be particularly valuable in tasks with more than two categories. However, it's important to note that calculating a weighted average ROC curve doesn't directly result in an actual ROC curve, but rather a weighted average AUC (Area Under the Curve) value. Each ROC curve represents the relationship between the True Positive Rate and False Positive Rate at a certain threshold, so we can't directly compute a weighted average ROC curve. In practice, we usually draw individual ROC curves for each class and then calculate a weighted average AUC value. This AUC value can be considered a comprehensive measure of the performance of all the ROC curves across classes.

AUC-ROC is considered a more advanced metric as it provides a more comprehensive view of model performance across all possible classification thresholds, rather than a single point estimate like accuracy, precision or recall. This means that it takes into account both the model's sensitivity (recall) and its specificity (precision), and doesn't require selecting a specific threshold in advance. AUC is particularly useful in imbalanced datasets, where a model's ability to distinguish between classes is more informative than its overall accuracy.

In conclusion, by using the metrics of accuracy, cross-entropy loss, precision, recall, F1 score and AUC-ROC, we can gain a more complete and nuanced understanding of the performance of our models. These metrics allow for a comprehensive comparison and evaluation of the models, capturing different aspects of the model's performance and enabling us to select the most suitable model for our task. These metrics are crucial for both binary and multi-category classification tasks, where different aspects of model performance need to be considered.

In [ ]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_state(self):
        self.precision.reset_states()
        self.recall.reset_states()

    def get_config(self):
        return super().get_config()

    @classmethod
    def from_config(cls, config):
        return cls(**config)


class MultiCategoryF1Score(tf.keras.metrics.Metric):
    def __init__(self, num_category=4, average='weighted', name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_category
        self.average = average
        self.confusion_matrix = self.add_weight(name="confusion_matrix", shape=(num_classes, num_classes), initializer="zeros", dtype=tf.float32)


    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.argmax(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        current_confusion_matrix = tf.math.confusion_matrix(y_true, y_pred, num_classes=self.num_classes, dtype=tf.float32)
        self.confusion_matrix.assign_add(current_confusion_matrix)

    def result(self):
        tp = tf.linalg.diag_part(self.confusion_matrix)
        fp = tf.reduce_sum(self.confusion_matrix, axis=0) - tp
        fn = tf.reduce_sum(self.confusion_matrix, axis=1) - tp

        precision_per_class = tp / (tp + fp + tf.keras.backend.epsilon())
        recall_per_class = tp / (tp + fn + tf.keras.backend.epsilon())

        if self.average == 'micro':
            precision = tf.reduce_sum(tp) / (tf.reduce_sum(tp) + tf.reduce_sum(fp))
            recall = tf.reduce_sum(tp) / (tf.reduce_sum(tp) + tf.reduce_sum(fn))
        elif self.average == 'macro':
            precision = tf.reduce_mean(precision_per_class)
            recall = tf.reduce_mean(recall_per_class)
        elif self.average == 'weighted':
            weights = tf.reduce_sum(self.confusion_matrix, axis=1)
            precision = tf.reduce_sum(precision_per_class * weights) / tf.reduce_sum(weights)
            recall = tf.reduce_sum(recall_per_class * weights) / tf.reduce_sum(weights)
        else:
            raise ValueError("Invalid average type. Accepted values are: 'micro', 'macro', 'weighted'")

        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_state(self):
        self.confusion_matrix.assign(tf.zeros_like(self.confusion_matrix))

    def get_config(self):
        config = super().get_config()
        config.update({"num_classes": self.num_classes, "average": self.average})
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


class MultiPrecision(MultiCategoryF1Score):
    def __init__(self, num_category=4, average='weighted', name='precision', **kwargs):
        super().__init__(num_category, average, name, **kwargs)

    def result(self):
        tp = tf.linalg.diag_part(self.confusion_matrix)
        fp = tf.reduce_sum(self.confusion_matrix, axis=0) - tp
        precision_per_class = tp / (tp + fp + tf.keras.backend.epsilon())
        if self.average == 'micro':
            precision = tf.reduce_sum(tp) / (tf.reduce_sum(tp) + tf.reduce_sum(fp))
        elif self.average == 'macro':
            precision = tf.reduce_mean(precision_per_class)
        elif self.average == 'weighted':
            weights = tf.reduce_sum(self.confusion_matrix, axis=1)
            precision = tf.reduce_sum(precision_per_class * weights) / tf.reduce_sum(weights)
        else:
            raise ValueError("Invalid average type. Accepted values are: 'micro', 'macro', 'weighted'")
        return precision


class MultiRecall(MultiCategoryF1Score):
    def __init__(self, num_category=4, average='weighted', name='recall', **kwargs):
        super().__init__(num_category, average, name, **kwargs)

    def result(self):
        tp = tf.linalg.diag_part(self.confusion_matrix)
        fn = tf.reduce_sum(self.confusion_matrix, axis=1) - tp
        recall_per_class = tp / (tp + fn + tf.keras.backend.epsilon())
        if self.average == 'micro':
            recall = tf.reduce_sum(tp) / (tf.reduce_sum(tp) + tf.reduce_sum(fn))
        elif self.average == 'macro':
            recall = tf.reduce_mean(recall_per_class)
        elif self.average == 'weighted':
            weights = tf.reduce_sum(self.confusion_matrix, axis=1)
            recall = tf.reduce_sum(recall_per_class * weights) / tf.reduce_sum(weights)
        else:
            raise ValueError("Invalid average type. Accepted values are: 'micro', 'macro', 'weighted'")
        return recall

In [ ]:
def binary_roc_curve(model, generator):
    y_true = generator.classes
    y_pred = model.predict(generator)
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)

    roc_auc = roc_auc_score(y_true, y_pred)  # 计算AUC值

    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

def categorical_roc_curve(model, generator):
    y_true = np.array(generator.classes)
    n_classes = len(np.unique(y_true))
    y_true_bin = label_binarize(y_true, classes=np.arange(n_classes))
    y_pred_bin = model.predict(generator)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    weights = []

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_bin[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        weights.append(np.sum(y_true == i))

    weighted_roc_auc = np.average(list(roc_auc.values()), weights=weights)

    plt.figure()
    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f) for label %i' % (roc_auc[i], i))

    plt.plot([0, 1], [weighted_roc_auc]*2, 'b--', label='Weighted Avg ROC curve (area = %0.2f)' % weighted_roc_auc)

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic for Multi-Class')
    plt.legend(loc="lower right")
    plt.show()

---
# Model Implementation

In this task, we need to make predictions for two values: whether the cell image is cancerous and the cell type, which is a classification task. I will use artificial neural networks to model these predictions. According to the requirements, I will use a simple MLP as the baseline model and a convolutional neural network (CNN) as the improved model.

## MLP Baseline Model for Binary Classification
In this task, we need to predict whether an image is cancerous, which is a binary classification task. I will use a simple Multilayer Perceptron (MLP) model to model this task, in order to determine the limitations of the model and compare its performance with the improved model. An MLP is an artificial neural network that consists of multiple layers of perceptrons or neurons, with each layer being fully connected to the next. These layers are typically categorized into three types: input layer, hidden layer(s), and output layer.

We can start with the following model to preprocess the image. Since the MLP model cannot directly handle image data, we need to flatten it into a 1-dimensional vector. Next is the hidden layer, where we can choose multiple hidden layers with different numbers of neurons depending on the complexity of the problem. Finally, the output layer will have only one neuron since our task is binary classification. It will output a value indicating the probability of the image being cancerous.

According to the requirements, we need to complete the baseline model and also attempt to build an improved model using other techniques. The baseline model will be used to observe the performance differences between the models.

Since we have a binary classification task, we will use binary cross-entropy loss as the loss function.

### Problem Formulation
For this task, the loss function uses binary cross-entropy, and the objective function can be formulated as follows:

$$\min_{h \in \mathcal{H}} \frac{1}{n} \sum_{i=1}^{n} -[y_i \log(h(\mathbf{x}_i)) + (1-y_i) \log(1-h(\mathbf{x}_i))]$$

Here, $\mathcal{H}$ is the hypothesis space, which represents the set of all possible MLP functions, $h(\mathbf{x}_i)$ is the prediction of the MLP for input $\mathbf{x}_i$, and $y_i$ is the true label (0 or 1).

For the hypothesis space $\mathcal{H}$, since MLP involves more complex forms of functions, we consider $\mathcal{H}$ as a function space consisting of all possible MLP models with various parameters, layers, and node numbers.

### Model Description
For the binary classification task of determining whether the cell is cancerous, the input layer will flatten the input image into a 1-dimensional tensor with a size of (27, 27, 3). It will then enter a fully connected layer with 512 neurons and use the ReLU activation function. The output layer will have 1 neuron and use the sigmoid activation function to obtain the probability of cancer classification. Since we are building a simple baseline model, we will use only one hidden layer. The number of perceptrons in the hidden layer is set to 512. It is important to note that the number of perceptrons in the hidden layer, the number of hidden layers, and the choice of activation function are hyperparameters that need to be adjusted. For a 27x27x3 image, the input layer has 2187 neurons. In this case, using a hidden layer with 512 neurons is considered a reasonable choice. This number is large enough to capture complex patterns in the input data while also being small enough to reduce the risk of overfitting. Additionally, 512 is a power of 2, which can improve computational efficiency in many deep learning frameworks.

In [ ]:
input_shape = (27, 27, 3)
baseline_binary = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
baseline_binary.summary()

### 编译模型
As we mentioned earlier, we are using the binary cross-entropy loss function for this task. Adam is chosen as the optimizer, which already includes momentum to prevent getting stuck in local minima. Additionally, Adam also incorporates learning rate decay, allowing the model to fine-tune its parameters more effectively in the later stages of training. We are using the default learning rate of 0.001, and we want to observe the performance of the baseline model in order to adjust the learning rate in subsequent models.

Simultaneously, we will be recording the loss, accuracy, precision, recall, and F1 score. This will enable us to track the performance changes of the model during training.

In [ ]:
baseline_binary.compile(loss='binary_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=['accuracy',
                                 tf.keras.metrics.Precision(name='precision'),
                                 tf.keras.metrics.Recall(name='recall'),
                                 F1Score()])

### Training the Model
The training cycle is also a hyperparameter that we need to adjust.

In [ ]:
epochs = 100
history_baseline_binary = baseline_binary.fit(
    train_isCancerous_generator,
    steps_per_epoch=len(train_data) // batch_size,
    epochs=epochs,
    validation_data=val_isCancerous_generator,
    validation_steps=len(val_data) // batch_size
)

Visualizing the training process

In [ ]:
epochs_range = range(1, epochs + 1)
metrics = [key for key in history_baseline_binary.history.keys() if not key.startswith("val_")]
_, axes = plt.subplots(1, 5, figsize=(20, 4))
for idx, metric in enumerate(metrics):
    col = idx
    axes[col].plot(epochs_range, history_baseline_binary.history[metric], label=f'Training {metric.capitalize()}')
    axes[col].plot(epochs_range, history_baseline_binary.history[f"val_{metric}"], label=f'Validation {metric.capitalize()}')
    axes[col].legend()
    axes[col].set_title(f'Training and Validation {metric.capitalize()}')
    axes[col].set_xlabel('Epoch')
    axes[col].set_ylabel(metric.capitalize())
plt.tight_layout()
plt.show()

Performance on unseen data

In [ ]:
baseline_binary.evaluate(test_isCancerous_generator)

ROC Curve and AUC Value

In [ ]:
binary_roc_curve(baseline_binary, test_isCancerous_generator)

### 总结模型性能
根据给出的性能指标，我们可以对模型的性能进行如下分析：
二分类基线模型似乎在训练集上表现良好，准确性、精确度、召回率和 F1 分数都相对较高。但是，还必须考虑验证集的结果，以查看二分类基线模型是否能够很好地泛化到未见数据。
然而，训练和验证结果之间似乎存在明显的差距，这是过度拟合的迹象。二分类基线可能对训练数据的学习太好，因此无法推广到新的、看不见的数据。
此外，另一个潜在的问题是模型的查全率（recall）和查准率（precision）在训练和验证阶段都表现非常不稳定。这可能表明模型在分类正例和反例时可能存在困难，这说明仍需要进一步优化以提高其泛化能力和收敛稳定性。
我们还注意到验证集的损失似乎在波动并且没有显示出一致的下降趋势，这又可能表明过度拟合并且后期的的学习率可能需要调整。
从ROC曲线来看，模型已经具备一定的分类能力，其AUC值达到0.94，表明模型在区分正负样本上的能力较强。然而，理想的ROC曲线应该更靠近图像的左上角，这意味着模型可以在保持高召回率的同时，尽可能降低假阳性率。因此，虽然现有模型的表现已经相对不错，但还是存在一定的提升空间。

### Improvement
这里我们将在前面所有内容的基础上总结一下对Baseline模型可能改进的探索。首先根据EDA我们发现了数据不平衡的问题，之前我们在分割数据的部分我通过分配类别权重来抑制不平衡数据集的影响。其次，根据我们基线模型的表现，我们发现了模型是过拟合的，它无法在验证集和不可见测试集上体现出泛化能力。因此，我们需要通过改进模型，添加正则化以及Dropout层等方式来解决过拟合的问题。并且批归一化层也会被使用，它同样可以增加模型的泛化能力的同时加速模型收敛速度。同时，我们将在提升模型中使用图像增强技术，这项技术可以增多训练数据从而降低过拟合的风险。第三，我们还发现基线模型收敛并不稳定，所以我们将降低学习率并且根据验证集损失值的变化动态降低学习率，从而保证模型稳定的收敛。

## MLP Baseline Model for Multi-class Task
In this task, we need to predict the class of cell images, making it a multi-class classification problem. We will still use a simple MLP model as our baseline model. The main difference from the MLP model for binary classification is the activation function in the output layer, which changes from sigmoid to softmax. Additionally, we will modify our loss function from binary cross-entropy to categorical cross-entropy, as our task has transitioned from binary to multi-class.

### Problem Formulation
For multi-class classification using a multilayer perceptron (MLP), the loss function employed is categorical cross-entropy. The objective function can be expressed as:

$$\min_{h \in \mathcal{H}} \frac{1}{n} \sum_{i=1}^{n} \left(-\sum_{c=1}^{C} y_{ic} \log(h_{ic}(\mathbf{x}_i))\right)$$

Here, $\mathcal{H}$ represents the hypothesis space, which is the set of all possible MLP functions. $h_{ic}(\mathbf{x}_i)$ denotes the predicted value (class $c$) by the MLP for input $\mathbf{x}_i$, and $y_{ic}$ corresponds to the one-hot encoding of the true label (value for class $c$).

Similar to the explanation given earlier, the hypothesis space $\mathcal{H}$ can be considered as the function space consisting of all possible MLP models with varying parameters, layers, and nodes.

### Model Description
The model structure remains similar to the binary classification baseline model.
Compared to the binary baseline model, we change the activation function in the output layer to softmax to handle multi-class classification.

In [ ]:
num_classes = 4
baseline_categorical = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

### Compiling the Model
For the multi-class task, we will use cross-entropy loss as the loss function for this training. We will continue to use Adam as the optimizer. Additionally, we will record the same metrics as in the binary classification baseline model. Precision, Recall, and F1 Score will be calculated using the weighted approach, which is applicable to multi-class problems.

In [ ]:
baseline_categorical.compile(loss='categorical_crossentropy',
                             optimizer=tf.keras.optimizers.Adam(),
                             metrics=['accuracy',
                                      MultiPrecision(),
                                      MultiRecall(),
                                      MultiCategoryF1Score(num_category=4)])

### Training the Model

In [ ]:
epochs = 100
history_baseline_categorical = baseline_categorical.fit(
    train_cellType_generator,
    steps_per_epoch=len(train_data) // batch_size,
    epochs=epochs,
    validation_data=val_cellType_generator,
    validation_steps=len(val_data) // batch_size
)

Visualizing the training process

In [ ]:
epochs_range = range(1, epochs + 1)
metrics = [key for key in history_baseline_categorical.history.keys() if not key.startswith("val_")]
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for idx, metric in enumerate(metrics):
    col = idx
    axes[col].plot(epochs_range, history_baseline_categorical.history[metric], label=f'Training {metric.capitalize()}')
    axes[col].plot(epochs_range, history_baseline_categorical.history[f"val_{metric}"], label=f'Validation {metric.capitalize()}')
    axes[col].legend()
    axes[col].set_title(f'Training and Validation {metric.capitalize()}')
    axes[col].set_xlabel('Epoch')
    axes[col].set_ylabel(metric.capitalize())
plt.tight_layout()
plt.show()

Performance on unseen data

In [ ]:
baseline_categorical.evaluate(test_cellType_generator)

ROC Curve and AUC Value

In [ ]:
categorical_roc_curve(baseline_categorical, test_cellType_generator)

### 总结模型性能
多分类基线模型似乎在训练集上表现欠佳，准确性、精确度、召回率和 F1 分数都相对较低。并且在验证集集上的性能表现更差。
与此同时，我们可以从损失的变化图中发现，训练和验证结果之间似乎存在明显的差距，这是过度拟合的迹象。
与二分类基线模型类似，验证集的损失似乎在波动并且下降趋势非常缓慢。
我们可以从准确度，precision，recall和f1_score的趋势图中看出，模型在后期时仍然不能稳定收敛。
此外，模型在不可见测试的表现也让人觉得的差强人意。在不可见测试集上f1分数只有大约0.71。
最后，从ROC曲线来看，我们的模型在不同类别上的表现有所差异。特别是最好一个类别，模型的AUC值为0.77，这相对于其他类别来说较低，说明模型在处理这个类别的任务上存在一定的困难。
模型的加权平均AUC为0.89，这是一个较好的分数。然而，我们也需要注意到，模型在处理第四个类别的任务上存在一定的挑战。我们将在提升模型中进一步优化模型，尤其是在处理第四个类别的任务上，以提高模型的整体性能。

### Improvement
对于改进策略与二分类模型的改进策略类似。

---
# Enhanced Model
To improve the model, we will use Convolutional Neural Networks (CNNs). CNNs are a specialized type of neural network primarily used for processing data with a grid-like structure, such as images. The distinguishing feature of CNNs is their hierarchical structure and weight-sharing mechanism. Each layer of a CNN consists of multiple convolutional filters, and each filter learns different features from the input data. This enables CNNs to capture local spatial correlations in the data. By sharing weights, CNNs reduce the number of parameters in the model, which not only reduces computational complexity but also helps prevent overfitting.

## Image Augmentation and Data Generator
As we said before, In order to improve the baseline model's performance and increase its ability to generalize, we introduce image augmentation techniques. These techniques apply random transformations to the images, such as rotation, shifting, shearing, and flipping, which can help the model learn more robust features and reduce overfitting.

To achieve this, we have enhanced the functionality of the existing `ImageDataGenerator` class provided by Keras. Our custom data generator, called `CustomImageDataGenerator`, randomly selects a specified number of augmentations each time an image is processed. By introducing this approach, we introduce more diversity into the training data, thereby enhancing the model's ability to generalize to unseen examples. The number of augmentations chosen has become a hyperparameter that requires tuning. We experimented with different values, including 3, 4, and 5, and observed that using 4 augmentations yielded better generalization compared to using 3. However, increasing the number to 5 did not bring significant improvements over 4.

In the `CustomImageDataGenerator` class, we override the `apply_transform` method. This method randomly selects `num_augmentations` transformations from a list of possible augmentations each time an image is processed.

Next, we instantiate this custom data generator and specify the types of transformations and their ranges. In this case, we allow the images to be rotated, shifted in width and height, sheared, zoomed, brightness adjusted, and flipped horizontally and vertically. The `fill_mode` parameter is set to 'nearest', which fills in any new pixels created by the transformations with their nearest neighbors. So we choose 4 as the hyperparameter for the number of augmentations.

Finally, we create the data generators for training the 'isCancerous' and 'cellType' models. These generators will read the images from the disk and apply the specified augmentations on the fly.
Through the use of this custom image data generator, we aim to increase the diversity of the training data, and thus improve the model's ability to generalize to unseen data.

In [ ]:
class CustomImageDataGenerator(tf.keras.preprocessing.image.ImageDataGenerator):
    def __init__(self, num_augmentations=3, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_augmentations = num_augmentations

    def apply_transform(self, x, transform_parameters):
        augmentations = [
            "theta",
            "tx",
            "ty",
            "shear",
            "zx",
            "zy",
            "flip_horizontal",
            "flip_vertical",
            "brightness",
        ]

        selected_augmentations = random.sample(augmentations, self.num_augmentations)
        filtered_transform_parameters = {
            key: value for key, value in transform_parameters.items() if key in selected_augmentations
        }

        return super().apply_transform(x, filtered_transform_parameters)

train_datagen = CustomImageDataGenerator(
    num_augmentations=4,
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.2,1.0],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

train_isCancerous_generator = train_datagen.flow_from_dataframe(
    train_data,
    directory='./Image_classification_data/patch_images',
    x_col='ImageName',
    y_col='isCancerous',
    target_size=(27, 27),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

train_cellType_generator = train_datagen.flow_from_dataframe(
    train_data,
    directory='./Image_classification_data/patch_images',
    x_col='ImageName',
    y_col='cellType',
    target_size=(27, 27),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

## Learning Rate Scheduler
As mentioned earlier, to ensure stable convergence in the later stages of training, we employ a callback method called `ReduceLROnPlateau` to dynamically adjust the learning rate. This method monitors the validation loss (`monitor='val_loss'`) and if the validation loss does not improve for consecutive 3 epochs (`patience=3`), the current learning rate is multiplied by a factor (`factor=np.sqrt(0.1)`) to decrease the learning rate. Additionally, this method sets a minimum value for the learning rate (`min_lr=0.5e-15`) to prevent the learning rate from becoming too small. This strategy helps us achieve more stable convergence in the later stages of training because as the training progresses, we require finer steps to optimize the model parameters.

In [ ]:
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), cooldown=0, patience=3, min_lr=0.5e-15)

## Setting Regularization Strength
Currently, I have selected 0.01 as the regularization strength coefficient, indicating that we have introduced a relatively mild regularization. We can also try adjusting other regularization strength coefficients to see their impact on the model's performance.

In [ ]:
l2_coeff = 0.01

## Binary Classification Convolutional Neural Network (Enhanced Model)
In this task, we still aim to predict whether a cell image is cancerous or not. I will use Convolutional Neural Networks (CNNs) as our enhanced model. CNNs have been widely adopted for image recognition tasks due to their excellent performance. Unlike the previous baseline model, our CNN model will leverage convolutional layers to capture local features in the image that may be overlooked by fully connected networks.

The enhanced model consists of multiple convolutional layers, max-pooling layers, a fully connected layer, and an output layer. The convolutional layers are responsible for extracting features from the input images, while the max-pooling layers reduce the spatial dimensions of the output from the convolutional layers, reducing model complexity and potential overfitting. The fully connected layer integrates these features, and the output layer provides the predicted probabilities for each class.

To prevent overfitting and improve model performance, we will also employ batch normalization and Dropout. Batch normalization accelerates the convergence speed of the model and acts as a form of regularization. Dropout randomly drops out a portion of neurons during training, enhancing the model's generalization ability.

Additionally, we will incorporate the data augmentation generator we just created to increase sample diversity during model training, further improving the model's generalization ability and performance.

Since the task remains the same, we will continue to use binary cross-entropy as the loss function.

### Problem Formulation
The objective function remains the same as the binary classification baseline model.

However, for the hypothesis space $\mathcal{H}$, we consider $\mathcal{H}$ as the function space consisting of all possible CNN models with varying parameters, layers, kernel sizes, and numbers.

### Model Description
Compared to the baseline model, we have made several changes.

Our binary classification CNN model includes convolutional layers, max-pooling layers, fully connected layers, and Dropout layers. Here's a detailed description of each layer in the model:

The model starts with two Convolutional Blocks. Each Convolutional Block consists of two Conv2D layers, two BatchNormalization layers, and a pooling layer. To enhance the expressive power and generalization performance of the model, we use two convolutions within each Convolutional Block. Batch normalization is inserted between convolutions and between convolution and pooling. The first Convolutional Block uses 32 3x3-sized filters, while the second Convolutional Block uses 64 3x3-sized filters. Due to the use of pooling operations, we apply the `same` padding parameter in the convolutional layers to maintain the same output feature map size as the input feature map.

The model then enters the Flatten layer, which flattens the output tensor from the convolutional layers into a 1D tensor. Next, the model enters the fully connected layers. Our binary classification CNN model includes two fully connected layers, each with 512 neurons and a BatchNormalization layer. A Dropout layer is added after each fully connected layer to mitigate overfitting. Finally, the model outputs a probability value representing the likelihood of the input belonging to the positive class.

We will use `relu` as the activation function throughout the model. L2 regularization will also be applied. It's important to note that the choice of activation function, regularization method, and regularization strength are hyperparameters that can be adjusted.

In [ ]:
model_binary = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_coeff)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2, seed=seed_value),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_coeff)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2, seed=seed_value),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_binary.summary()

### Compiling the Model
Similar to the binary classification baseline model, but with a difference in the learning rate adjustment. Based on the results of the baseline model, we need to decrease the learning rate to ensure stable convergence.

In [ ]:
model_binary.compile(loss='binary_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                     metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall'),
                       F1Score()])

### Training the Model

In [ ]:
epochs = 100
history_binary = model_binary.fit(
    train_isCancerous_generator,
    steps_per_epoch=len(train_data) // batch_size,
    epochs=epochs,
    validation_data=val_isCancerous_generator,
    validation_steps=len(val_data) // batch_size,
    class_weight=class_weight_dict_isCancerous
    , callbacks=[lr_reducer]
)

Visualizing the training process

In [ ]:
epochs_range = range(1, epochs + 1)
metrics = [key for key in history_binary.history.keys() if not key.startswith("val_") and not key.startswith("lr")]
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for idx, metric in enumerate(metrics):
    col = idx
    axes[col].plot(epochs_range, history_binary.history[metric], label=f'Training {metric.capitalize()}')
    axes[col].plot(epochs_range, history_binary.history[f"val_{metric}"], label=f'Validation {metric.capitalize()}')
    axes[col].legend()
    axes[col].set_title(f'Training and Validation {metric.capitalize()}')
    axes[col].set_xlabel('Epoch')
    axes[col].set_ylabel(metric.capitalize())
plt.tight_layout()
plt.show()

Performance on unseen data

In [ ]:
model_binary.evaluate(test_isCancerous_generator)

ROC Curve and AUC Value

In [ ]:
binary_roc_curve(model_binary, test_isCancerous_generator)

### 总结模型性能
根据训练过程的曲线来看，我们可以发现模型的收敛与二分类基线模型相比稳定非常很多，例如损失函数的曲线。即使召回率的收敛在前期可能并不稳定，但其后期也保证了稳定收敛。并且其他指标也可以证明其稳定收敛。此外，我们可以发现验证集和训练集之间的差距也非常下，这说明模型在保证性能的同时保证了其泛化能力。与此同时，模型在不可见测试集下得到了非常高的分数，模型的准确度大约为0.94，precision大约为0.91，recall大约为0.93并且F1分数大约为0.92。这说明模型在区分图像是否为癌症的能力上表现得非常出色。我们可以通过recall值知道，模型可以很好的找到真正例，即尽可能找出所有的真实的癌症病例，这对于癌症检测来说是非常有帮助的。同时，它的ROC曲线非常接近左上角，表明模型在分类任务中具有很好的灵敏度和特异性，AUC值高达0.98，表明模型对于任意一张图像，正确分类的概率是非常高的。整体而言，相较于基线模型，这个二分类CNN模型在本任务上的表现有了显著的提升，更好地利用了图像的空间信息，减少了误诊和漏诊的可能性。

## Multi-class Convolutional Neural Network (Enhanced Model)
In this task, our goal remains to predict the classification of cell images. We will continue to use Convolutional Neural Networks (CNNs) as our enhanced model.

As mentioned in the enhanced model for binary classification, techniques such as regularization and dropout layers are employed to prevent overfitting.

Since this is a multi-class task, we will use the categorical cross-entropy function as our loss function.

### Problem Formulation
The objective function remains the same as the multi-class baseline model.

However, for the hypothesis space $\mathcal{H}$, we consider $\mathcal{H}$ as the function space consisting of all possible CNN models with varying parameters, layers, kernel sizes, and numbers.

### Model Description
The structure of the multi-class CNN model is almost identical to the binary classification CNN model. However, as mentioned earlier, the number of layers and the number of perceptrons are hyperparameters that we need to adjust. Based on our experiments, we found that removing two batch normalization layers from the fully connected layers in the CNN multi-class model leads to more stable convergence and slightly better performance on unseen test data. Additionally, since this task is a multi-class classification, we change the activation function in the final output layer from sigmoid to softmax and set the number of classes to 4.

In [ ]:
model_categorical = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_coeff)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_coeff)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
model_categorical.summary()

### Compiling the Model

In [ ]:
model_categorical.compile(loss='categorical_crossentropy',
                          optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
                          metrics=['accuracy',
                                   tf.keras.metrics.Precision(name='precision'),
                                   tf.keras.metrics.Recall(name='recall'),
                                   MultiCategoryF1Score()])

### Training the Model

In [ ]:
epochs = 100
history_categorical = model_categorical.fit(
    train_cellType_generator,
    steps_per_epoch=len(train_data) // batch_size,
    epochs=epochs,
    validation_data=val_cellType_generator,
    validation_steps=len(val_data) // batch_size,
    class_weight=class_weight_dict_cellType
    , callbacks=[lr_reducer]
)

Visualizing the training process

In [ ]:
epochs_range = range(1, epochs + 1)
metrics = [key for key in history_categorical.history.keys() if not key.startswith("val_") and not key.startswith("lr")]
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for idx, metric in enumerate(metrics):
    col = idx
    axes[col].plot(epochs_range, history_categorical.history[metric], label=f'Training {metric.capitalize()}')
    axes[col].plot(epochs_range, history_categorical.history[f"val_{metric}"], label=f'Validation {metric.capitalize()}')
    axes[col].legend()
    axes[col].set_title(f'Training and Validation {metric.capitalize()}')
    axes[col].set_xlabel('Epoch')
    axes[col].set_ylabel(metric.capitalize())
plt.tight_layout()
plt.show()

Performance on unseen data

In [ ]:
model_categorical.evaluate(test_cellType_generator)

ROC Curve and AUC Value

In [ ]:
categorical_roc_curve(model_categorical, test_cellType_generator)

### 总结模型性能
观察训练过程中的曲线，我们可以发现改进的CNN多分类模型表现出显著的稳定性，即使在50轮前有较为明显的波动，但50轮后所有训练过程曲线都趋于稳定。值得注意的是验证集和训练集的损失有加大差距，这是由于分配权重所导致的。在训练过程中，这些权重会放大某些类别的损失值，以便模型能够更好地学习这些类别。与此同时，我们可以从训练过程的曲线或训练集和验证集的分数中，看出模型在训练集和验证集之间的性能差距很小，显示出良好的泛化能力。
在未见过的测试集上，模型取得了很高的分数，准确度和F1分数都达到了大约0.81，精确度大约为0.84已经召回率大约为0.79。这些高分表明模型在区分多个类别的图像上做得非常出色。
模型对每个类别的分类能力也非常强，各类别的AUC值分别为0.95，0.96，0.98，0.91，而加权平均AUC为0.96。特别是其ROC曲线接近左上角，显示出模型在分类任务中具有高度的灵敏度和特异性。与多分类基线模型相比，每个类别的AUC值都有提升，特别是最后一个类别从0.77提升到0.91，这说明该改进的多分类模型在识别不同类别，特别是对于最后一个类别的识别能力上，有了显著的提升。
总的来说，相较于基线模型，改进的多分类模型在本任务上的表现有了显著的提升。

# Enhanced Multi-class Convolutional Neural Network (DI/HD Part)
According to the task requirements, we need to enhance the multi-class model using the `data_labels_extraData.csv` file. First, let's load this dataset and examine its characteristics. We can see that the dataset contains the `isCancerous` label, but there is no `cellType` column. Therefore, we don't have the corresponding multi-class labels. Now we have one dataset with multi-class labels and another dataset without multi-class labels. Hence, semi-supervised learning becomes a natural choice. Semi-supervised learning is a machine learning approach that leverages both labeled data and unlabeled data for training.

The main idea of semi-supervised learning is to utilize the distribution information of the unlabeled data to assist the classification decisions of labeled data, thereby improving the model's performance. Specifically, it assumes that the decision boundaries between classes should pass through regions with low data density, or in other words, similar data samples should have the same or similar output labels.

In our problem, we can utilize the improved multi-class model trained on labeled data and then use this model to make predictions on the dataset without multi-class labels, obtaining predicted labels. Then, we can treat these predicted labels as new labels and combine them with the original labels for training the model again. Through this iterative process, we can gradually improve the model's prediction accuracy on the unlabeled data, thereby enhancing the overall model performance.

We will employ a semi-supervised learning algorithm called FixMatch ([Sohn, 2020](https://arxiv.org/abs/2001.07685)) to accomplish our task.

In [ ]:
data_extra = pd.read_csv('./Image_classification_data/data_labels_extraData.csv')
data_extra['isCancerous'] = data_extra['isCancerous'].astype(str)
data_extra.info()

## FixMatch Implementation
FixMatch is a semi-supervised learning algorithm proposed by the Google Brain team in 2020, and it has demonstrated excellent performance in various image recognition benchmarks. The main characteristics of FixMatch are its simplicity and effectiveness, successfully integrating some key principles of semi-supervised learning into a framework that is easy to understand and implement.

The workflow of FixMatch can be summarized into the following steps:
- First, apply a series of mild augmentations (such as random cropping and horizontal flipping) to the input images to generate weakly augmented versions.
- Second, use the current trained model to make predictions on the weakly augmented images, generating pseudo-labels.
- Third, apply a series of more aggressive augmentations (such as changes in color, contrast, and brightness) to the same input images to generate strongly augmented versions. The authors of FixMatch employ techniques such as Cutout ([DeVries](https://arxiv.org/abs/1708.04552)), CTAugment ([Berthelot](https://arxiv.org/abs/1911.09785)), and RandAugment ([Cubuk](https://arxiv.org/abs/1909.13719)) for highly aggressive image augmentation.
- Fourth, calculate the consistency loss. The model is trained to produce consistent predictions on both the weakly augmented and strongly augmented images. This is typically achieved by minimizing the discrepancy between the predictions, computed using cross-entropy loss.
- Fifth, FixMatch introduces a probability threshold, where pseudo-labels are only used for training if the model's prediction on the weakly augmented image surpasses a certain probability threshold. This step helps prevent the model from relying too heavily on erroneous pseudo-labels.
- Finally, the model's final loss function is a combination of supervised loss (i.e., the model's performance on labeled images) and unsupervised loss (i.e., the consistency on weakly augmented and strongly augmented images). In this way, the model can leverage both labeled and unlabeled data simultaneously. The goal of FixMatch is to minimize the discrepancy between the model's predictions on weakly augmented and strongly augmented images while minimizing the prediction error on labeled images. This design enables FixMatch to effectively utilize unlabeled data, enhance the model's generalization capability, and reduce the risk of overfitting.

### Data Generator
As mentioned earlier, FixMatch introduces special image augmentation techniques, such as RandAugment. Therefore, to implement FixMatch reliably and effectively, we encapsulate the RandAugment method implemented in the imgaug library into an `ImageDataGenerator` to generate specially augmented image data. This allows us to meet the requirements of FixMatch while leveraging all the advantages of the `ImageDataGenerator`.

In [ ]:
class RandAugmentGenerator(tf.keras.preprocessing.image.ImageDataGenerator):
    def __init__(self, n, m, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.rand_aug = iaa.RandAugment(n=n, m=m)

    def apply_transform(self, x, transform_parameters):
        x = tf.cast(x, tf.uint8)
        x_aug = self.rand_aug(image=x.numpy())
        x_aug = tf.cast(x_aug, tf.float64)
        return super().apply_transform(x_aug, transform_parameters)

### Image Augmentation
The following code snippet bears some similarity to the image augmentation method we created earlier for the enhanced model. However, the difference lies in the fact that here we augment the extra dataset to meet the specific requirements of weakly augmented and strongly augmented images in the FixMatch algorithm. For weak augmentation, we perform mild transformations such as horizontal flipping and vertical flipping, and scale the pixel values to the range of [0, 1]. For strongly augmented images, we use the previously encapsulated `RandAugmentGenerator` class to augment the images.

Additionally, we want all samples from the extra dataset to be processed by the model within one training epoch. Therefore, we need to calculate the batch size for the extra dataset accordingly. To compute the batch size for the extra dataset, we divide the total number of samples in the extra dataset by the total number of samples in the training dataset and then multiply by the original batch size. This calculation ensures that all samples from the extra dataset are processed by the model in each training epoch, thereby improving the learning efficiency and performance of the model.

In [ ]:
unlabeled_batch_size = len(data_extra) // len(train_data) * batch_size
unlabeled_datagen_weak = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
)

unlabeled_generator_weak = unlabeled_datagen_weak.flow_from_dataframe(
    data_extra,
    directory='./Image_classification_data/patch_images',
    x_col='ImageName',
    y_col='isCancerous',
    target_size=(27, 27),
    batch_size=unlabeled_batch_size,
    class_mode='binary',
    shuffle=False
)

unlabeled_datagen_strong = RandAugmentGenerator(
    n=4, m=6,
    rescale=1./255,
    fill_mode='nearest'
)

unlabeled_generator_strong = unlabeled_datagen_strong.flow_from_dataframe(
    data_extra,
    directory='./Image_classification_data/patch_images',
    x_col='ImageName',
    y_col='isCancerous',
    target_size=(27, 27),
    batch_size=unlabeled_batch_size,
    class_mode='binary',
    shuffle=False
)

### Problem Formulation
For the FixMatch approach we are using, the loss function consists of two parts: supervised loss $\(\ell_s\)$ and unsupervised loss $\(\ell_u\)$. Therefore, our objective function can be formulated as follows:

$$\min_{h \in \mathcal{H}} \ell_s(h; \mathbf{x}_l, y_l) + \lambda \cdot \ell_u(h; \mathbf{x}_u)$$

Here, $\(\mathcal{H}\)$ is the hypothesis space, which is the set of all possible neural network models. $\(h\)$ represents a specific model; $\(\mathbf{x}_l, y_l\)$ and $\(\mathbf{x}_u\)$ represent the labeled input data, labeled output data, and unlabeled input data, respectively; $\(\lambda\)$ is a hyperparameter that balances the importance of different losses.

- The supervised loss $\(\ell_s(h; \mathbf{x}_l, y_l)\)$ is the supervised loss of the model $\(h\)$ on the labeled data $\((\mathbf{x}_l, y_l)\)$.

$$\ell_s=\frac{1}{B} \sum_{b=1}^B \mathrm{H}\left(p_b, p_{\mathrm{m}}\left(y \mid \alpha\left(x_b\right)\right)\right)$$

Here, $\(B\)$ is the batch size, $\(p_b\)$ is the predicted probability distribution of the model $\(h\)$ for the input $\(x_b\)$, $\(p_{\mathrm{m}}\)$ is the probability distribution of the true label $\(y\)$, $\(\alpha\)$ is the data augmentation function, and $\(\mathrm{H}\)$ is the cross-entropy loss.

- The unsupervised loss $\(\ell_u(h; \mathbf{x}_u)\)$ is the unsupervised loss of the model $\(h\)$ on the unlabeled data $\(\mathbf{x}_u\)$, where the pseudo-labels of $\(\mathbf{x}_u\)$ are generated by the model $\(h\)$ itself.

$$\ell_{u}=\frac{1}{B} \sum_{b=1}^{B} \mathbb{1}\left(\max \left(p_b\right) \geq \tau\right) \mathrm{H}\left(\hat{q}_b, p_{\mathrm{m}}\left(y \mid \mathcal{A}\left(u_b\right)\right)\right)$$

Here, $\(q_b\)$ is the predicted probability distribution of the model $\(h\)$ for the input $\(u_b\)$, $\(\hat{q}_b\)$ is the probability distribution of the pseudo-labels corresponding to $\(q_b\)$, $\(\mathcal{A}\)$ is a stronger data augmentation function, $\(\tau\)$ is the confidence threshold, and the unsupervised loss for an unlabeled sample is only computed if the model's predicted probability for a certain class is greater than $\(\tau\)$.

By optimizing this objective function, we can perform supervised learning on labeled data and self-supervised learning on unlabeled data, fully utilizing the data to improve the model's performance.

In [ ]:
def apply_fixmatch(x, y_true, x_unlabeled, model, optimizer, training=True, class_weight_dict=None, consistency_weight=1, confidence_threshold=0.95):
    with tf.GradientTape() as tape:
        y_pred_labeled = model(x)
        # 计算有监督损失
        # 如果是训练并且给定了类别权重，则根据权重计算损失。
        if class_weight_dict is not None and training:
            class_weight_tensor = tf.constant([class_weight_dict[0], class_weight_dict[1], class_weight_dict[2], class_weight_dict[3]])
            sample_weights = tf.gather(class_weight_tensor, tf.argmax(y_true, axis=-1))
            supervised_loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true, y_pred_labeled) * sample_weights)
        else:
            supervised_loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true, y_pred_labeled))
        final_loss = supervised_loss

        # 计算模型在原始无标签数据和增强无标签数据上的输出
        if x_unlabeled is not None:
            # 对无标签数据进行数据增强
            y_pred_unlabeled_weak = model(x_unlabeled[0])
            y_pred_unlabeled_strong = model(x_unlabeled[1])
            # 计算伪标签
            pseudo_labels = tf.argmax(y_pred_unlabeled_weak, axis=-1)
            # 计算最大概率
            max_probs = tf.reduce_max(y_pred_unlabeled_weak, axis=-1)
            # Confidence-based masking 基于置信度的掩码
            mask = max_probs > confidence_threshold
            # 计算一致性损失
            pseudo_labels_one_hot = tf.one_hot(pseudo_labels, depth=tf.shape(y_pred_unlabeled_weak)[-1])
            per_example_loss = tf.keras.losses.categorical_crossentropy(pseudo_labels_one_hot, y_pred_unlabeled_strong)
            consistency_loss = tf.reduce_sum(per_example_loss * tf.cast(mask, per_example_loss.dtype)) / tf.reduce_sum(tf.cast(mask, tf.float32) + tf.keras.backend.epsilon())

            final_loss = final_loss + consistency_weight * consistency_loss

        # 计算准确度
        accuracy = tf.keras.metrics.CategoricalAccuracy()
        accuracy.update_state(y_true, y_pred_labeled)

        if training:
            # 反向传播和优化
            grads = tape.gradient(final_loss, model_categorical.trainable_variables)
            optimizer.apply_gradients(zip(grads, model_categorical.trainable_variables))

    return {"loss": final_loss,
            "supervised_loss": supervised_loss,
            "consistency_loss" :consistency_loss if x_unlabeled is not None else None,
            "accuracy": accuracy.result()}

### Training the Model
Since our model has been pre-trained earlier, to avoid disrupting the useful patterns learned, we choose a smaller learning rate for fine-tuning. Additionally, we incorporate a learning rate adjustment strategy where the learning rate is reduced when the model's performance on the validation set does not significantly improve. This helps to stimulate the learning process of the model and search for better solutions, which is different from our previous learning rate decay strategy. In each training epoch, we train and optimize the model by computing the supervised loss on labeled data and the consistency loss on unlabeled data. After training, we evaluate the model's performance on the validation set to assess its generalization capability. This way, we can leverage both labeled and unlabeled data within the framework of semi-supervised learning and improve the model's performance.

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-8)
epochs = 5
min_val_loss = float('inf')
num_epochs_no_improvement = 0
current_learning_rate = adam.learning_rate.numpy()
history = {"loss": [], "val_loss": [], "accuracy": [], "val_accuracy": [], "precision":[],
           "recall":[], "f1":[], "val_precision":[], "val_recall":[], "val_f1":[]}

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    total_loss = 0
    total_supervised_loss = 0
    total_consistency_loss = 0
    total_category_2_loss = 0
    total_accuracy = 0
    total_precision = 0
    total_recall = 0
    total_f1_score = 0
    steps = 0

    total_batches = math.ceil(len(train_data) / batch_size)
    with tqdm(total=total_batches, desc="Training", ncols=100) as progress_bar:
        for (x_batch, y_batch), (x_unlabeled_weak, _), (x_unlabeled_strong, _) in zip(train_cellType_generator, unlabeled_generator_weak, unlabeled_generator_strong):
            progress_bar.update(1)
            result = apply_fixmatch(x_batch, y_batch, (x_unlabeled_weak, x_unlabeled_strong),
                                    model_categorical, adam,
                                    class_weight_dict=class_weight_dict_cellType,
                                    confidence_threshold=0.95)
            total_loss += result["loss"]
            total_supervised_loss += result["supervised_loss"]
            total_consistency_loss += result["consistency_loss"]
            total_accuracy += result["accuracy"]
            steps += 1
            # 检查是否已经处理了所有批次
            if steps * batch_size >= len(train_data):
                break

        # 计算并打印平均损失和准确率
        avg_loss = total_loss / steps
        avg_supervised_loss = total_supervised_loss / steps
        avg_consistency_loss = total_consistency_loss / steps
        avg_accuracy = total_accuracy / steps

        history["loss"].append(avg_loss.numpy())
        history["accuracy"].append(avg_accuracy.numpy())
        print(f" - loss: {avg_loss.numpy():.4f}, supervised_loss: {avg_supervised_loss.numpy():.4f}, consistency_loss: {avg_consistency_loss.numpy():.4f}, accuracy: {avg_accuracy.numpy():.4f}")

    total_loss = 0
    total_accuracy = 0
    total_recall = 0
    total_precision = 0
    total_f1_score = 0
    steps = 0
    for x_batch, y_batch in val_cellType_generator:
        result = apply_fixmatch(x_batch, y_batch, None, model_categorical, adam, training=False)
        total_loss += result["loss"]
        total_accuracy += result["accuracy"]
        steps += 1

        if steps * batch_size >= len(test_data):
            break

    avg_loss = total_loss / steps
    avg_accuracy = total_accuracy / steps

    history["val_loss"].append(avg_loss)
    history["val_accuracy"].append(avg_accuracy.numpy())

    print(f"Validation set： - loss: {avg_loss.numpy():.4f}, accuracy: {avg_accuracy.numpy():.4f}， lr: {current_learning_rate:.2e}")

    # 检查是否需要更新学习率
    if avg_loss < min_val_loss:
        min_val_loss = avg_loss
        num_epochs_no_improvement = 0
    else:
        current_learning_rate *= np.sqrt(0.1)
        current_learning_rate = max(current_learning_rate, 0.5e-15)
        adam.learning_rate.assign(current_learning_rate)
        num_epochs_no_improvement = 0

Performance on unseen data

In [ ]:
model_categorical.evaluate(test_cellType_generator)

ROC Curve and AUC Value

In [ ]:
categorical_roc_curve(model_categorical, test_cellType_generator)

### 总结模型性能
从模型的训练和评估结果来看，使用FixMatch策略对改进的CNN多分类模型进行优化后，模型的表现有了进一步的提升。具体来说，模型的损失从0.8383降低到了0.8229，表明模型在预测错误时的惩罚减小了，这是模型性能提升的一个重要指标。而且，模型的准确率从0.7990提升到了0.8283，表示模型对图像类别预测的准确性有了显著的提升。

此外，模型的精确度从0.8296提升到了0.8461，召回率从0.7747提升到了0.8025，这表明模型在识别正样本的能力上有了显著的提高，同时也减少了对负样本的误识别。F1分数从0.7677提升到了0.8276，也验证了模型在精确度和召回率之间取得了更好的平衡。虽然ROC曲线没有明显变化，但这可能是因为模型对于各个类别的预测已经非常接近理想情况，进一步的优化空间较小。

总的来说，通过使用FixMatch策略，我们成功地提升了改进的CNN多分类模型的性能，进一步提高了模型对于图像分类任务的处理能力。